In [80]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
import math

In [81]:
child_number = pd.read_csv('botswana.tsv', sep = '\t')
child_number.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [82]:
set(child_number['religion'])
#child_number.shape

{'catholic', 'other', 'protestant', 'spirit'}

In [83]:
child_withoutna = child_number.dropna(how = 'any')
child_withoutna.shape

(1834, 15)

In [84]:
child_number['nevermarr'] = list(map(lambda x: 1 if math.isnan(x) else 0, child_number['agefm']))

In [85]:
child_number = child_number.drop('evermarr', axis = 1)
child_number.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [86]:
child_number['agefm']= list(map(lambda x: 0 if math.isnan(x) else x, child_number['agefm']))
child_number['agefm'].head()

0     0.0
1    20.0
2    22.0
3     0.0
4    24.0
Name: agefm, dtype: float64

In [87]:
child_number['heduc'] = list(map(lambda x,y : -1 if x==1 else y, child_number['nevermarr'],child_number['heduc']))
child_number['heduc'].head()

0    -1.0
1    14.0
2     1.0
3    -1.0
4    12.0
Name: heduc, dtype: float64

In [88]:
sum(child_number['heduc'].isnull()==True)

123

In [89]:
child_number['idlnchld_noans'] = list(map(lambda x: 1 if math.isnan(x) else 0, child_number['idlnchld']))
child_number['idlnchld'] = list(map(lambda x: -1 if  math.isnan(x) else x, child_number['idlnchld']))
child_number['heduc_noans'] = list(map(lambda x: 1 if  math.isnan(x) else 0, child_number['heduc']))
child_number['heduc'] = list(map(lambda x: -2 if  math.isnan(x) else x, child_number['heduc']))
child_number['usemeth_noans'] = list(map(lambda x: 1 if  math.isnan(x) else 0, child_number['usemeth']))
child_number['usemeth'] = list(map(lambda x: -1 if  math.isnan(x) else x, child_number['usemeth']))

In [90]:
child_withoutna = child_number.dropna(how = 'any', subset = ['knowmeth','electric','radio','tv','bicycle'])
child_withoutna.shape


(4348, 18)

In [91]:
child_withoutna.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [92]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + electric + urban + radio+ tv + bicycle + nevermarr+'\
             'idlnchld_noans + heduc_noans + usemeth_noans', data=child_withoutna)
fitt = m1.fit()
print (fitt.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        21:40:47   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [93]:
print ('Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitt.resid, fitt.model.exog)[1])

Breusch-Pagan test: p=0.000000


/home/anastasiia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


In [94]:
m1_hc= smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio+ tv + bicycle + nevermarr+'\
            'idlnchld_noans + heduc_noans + usemeth_noans', data=child_withoutna)
fitt= m1_hc.fit(cov_type='HC1')
print (fitt.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        21:40:51   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [95]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr+'\
             'idlnchld_noans + heduc_noans + usemeth_noans', data=child_withoutna)
fitt = m2.fit()
print (fitt.summary())


                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        21:40:54   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [96]:
print ('Breusch-Pagan test: p=%f' % sms.het_breushpagan(fitt.resid, fitt.model.exog)[1])

Breusch-Pagan test: p=0.000000


/home/anastasiia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `het_breushpagan` is deprecated, use `het_breuschpagan` instead!
Use het_breuschpagan, het_breushpagan will be removed in 0.9 
(Note: misspelling missing 'c')
  """Entry point for launching an IPython kernel.


In [97]:
m2_hc = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr+'\
             'idlnchld_noans + heduc_noans + usemeth_noans', data=child_withoutna)
fitt = m2_hc.fit(cov_type='HC1')
print (fitt.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        21:40:55   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [98]:
print ( m1_hc.fit().compare_f_test(m2_hc.fit()))

(0.9192357784629898, 0.4672305547275616, 5.0)


In [99]:
m3_hc = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban+ electric + bicycle + nevermarr+'\
             'idlnchld_noans + heduc_noans', data=child_number)
fitt = m3_hc.fit(cov_type='HC1')
print (fitt.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.6
Date:                Thu, 15 Feb 2018   Prob (F-statistic):               0.00
Time:                        21:41:07   Log-Likelihood:                -7827.0
No. Observations:                4349   AIC:                         1.568e+04
Df Residuals:                    4336   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1932      0.262     -4.

In [100]:
print ( m2_hc.fit().compare_f_test(m3_hc.fit()))

(61.931896071736595, 3.355342050990348e-39, 3.0)
